# 01. function

In [ ]:
suppressMessages(library(Seurat))
suppressMessages(library(SeuratObject))
suppressMessages(library(SeuratDisk))
suppressMessages(library(ggplot2))
suppressMessages(library(patchwork))
suppressMessages(library(dplyr))
suppressMessages(library(data.table))
suppressMessages(library(Matrix))
suppressMessages(library(rjson))
suppressMessages(library(RColorBrewer))
suppressMessages(library(Seurat))
suppressMessages(library(SeuratObject))
suppressMessages(library(SeuratDisk))
suppressMessages(library(ggplot2))
suppressMessages(library(patchwork))
suppressMessages(library(dplyr))
suppressMessages(library(data.table))
suppressMessages(library(Matrix))
suppressMessages(library(rjson))
suppressMessages(library(RColorBrewer))

### To use, modify inputfile and prefix, the output will be automatically saved in the current folder

gem_to_seuratObject <- function(gem, prefix, binsize){
    #' group counts into bins
    data = fread(file = gem)
    colnames(data) = c("geneID", "x", "y", "MIDCounts")
    data$x <- trunc(data$x / binsize) * binsize
    data$y <- trunc(data$y / binsize) * binsize
    # trunc function, only keeps the integer part
    # adjust coordinates based on binsize, e.g., if binsize = 10, x = 2401, y = 4522,
    # then after processing, x = 2400, y = 4520

    if ('MIDCounts' %in% colnames(data)) {
      data <- data[, .(counts = sum(MIDCounts)), by = .(geneID, x, y)]
    } else {
      data <- data[, .(counts = sum(UMICount)), by = .(geneID, x, y)]
    }

    #' create sparse matrix from stereo
    data$cell <- paste0(prefix, ':', data$x, '_', data$y)
    data$geneIdx <- match(data$geneID, unique(data$geneID))
    data$cellIdx <- match(data$cell, unique(data$cell))

    mat <- sparseMatrix(i = data$geneIdx, j = data$cellIdx, x = data$counts,
                        dimnames = list(unique(data$geneID), unique(data$cell)))
    # sparseMatrix function for creating a sparse matrix

    cell_coords <- unique(data[, c('cell', 'x', 'y')])
    rownames(cell_coords) <- cell_coords$cell

    seurat_spatialObj <- CreateSeuratObject(counts = mat, project = 'Stereo', assay = 'Spatial',
                                            names.delim = ':', meta.data = cell_coords)

    #' create pseudo image
    cell_coords$x <- cell_coords$x - min(cell_coords$x) + 1
    cell_coords$y <- cell_coords$y - min(cell_coords$y) + 1

    tissue_lowres_image <- matrix(1, max(cell_coords$y), max(cell_coords$x))
    # matrix(aa, x, y) creates a matrix with aa as input vector, x rows, y columns
    # construct a seurat image

    tissue_positions_list <- data.frame(row.names = cell_coords$cell,
                                        tissue = 1,
                                        row = cell_coords$y, col = cell_coords$x,
                                        imagerow = cell_coords$y, imagecol = cell_coords$x)

    scalefactors_json <- toJSON(list(fiducial_diameter_fullres = binsize,
                                     tissue_hires_scalef = 1,
                                     tissue_lowres_scalef = 1))
    # toJSON: convert json format to list format

    #' function to create image object
    generate_spatialObj <- function(image, scale.factors, tissue.positions, filter.matrix = TRUE){
      if (filter.matrix) {
        tissue.positions <- tissue.positions[which(tissue.positions$tissue == 1), , drop = FALSE]
      }

      unnormalized.radius <- scale.factors$fiducial_diameter_fullres * scale.factors$tissue_lowres_scalef
      spot.radius <- unnormalized.radius / max(dim(x = image))

      return(new(Class = 'VisiumV1',
                 image = image,
                 scale.factors = scalefactors(spot = scale.factors$tissue_hires_scalef,
                                              fiducial = scale.factors$fiducial_diameter_fullres,
                                              hires = scale.factors$tissue_hires_scalef,
                                              lowres = scale.factors$tissue_lowres_scalef),
                 coordinates = tissue.positions,
                 spot.radius = spot.radius))
    }

    spatialObj <- generate_spatialObj(image = tissue_lowres_image,
                                      scale.factors = fromJSON(scalefactors_json),
                                      tissue.positions = tissue_positions_list)
    # can be understood as constructing a spatial background

    #' import image into seurat object
    spatialObj <- spatialObj[Cells(x = seurat_spatialObj)]
    DefaultAssay(spatialObj) <- 'Spatial'

    seurat_spatialObj[['slice1']] <- spatialObj
    rm("spatialObj")
    rm("data")
    rm("mat")

    return(seurat_spatialObj)
}

calc_percenterage_feature_set <- function(sce){
    #01 calculate percent of mt, ribo, hb and plat each bin
    sce <- PercentageFeatureSet(sce, '^MT-', col.name = "percent.mt")
    sce <- PercentageFeatureSet(sce, '^RP[SL]', col.name = "percent.ribo")
    sce <- PercentageFeatureSet(sce, '^HB[^(P)]', col.name = "percent.hb")
    return(sce)
}

filter_st_dat <- function(sce, minFeature, maxFeature, minCount, maxCount, minCell, mtRatio){

    selected_c <- WhichCells(sce, expression = (nCount_Spatial > minCount & nCount_Spatial < maxCount & nFeature_Spatial > minFeature & nFeature_Spatial < maxFeature & percent.mt < mtRatio))
    selected_f <- rownames(sce)[Matrix::rowSums(sce) > minCell]
    sce <- subset(sce, features = selected_f, cells = selected_c)
}

heatmap_Palette <- colorRampPalette(rev(brewer.pal(11, 'Spectral')))
iPlot <- function(object, features, pt.size = 0.1, cluster_Palette = ColorPalette(50)){
    plot <- ggplot(object@meta.data, aes_string(x = 'x', y = 'y', color = features)) +
            geom_point(shape = 19, size = pt.size) +
            theme(axis.text = element_blank(), axis.ticks = element_blank(), panel.grid = element_blank(),
                  axis.title = element_blank(), axis.line = element_blank(), legend.position = 'right') +
            coord_fixed()
    if (features %in% c('nCount_Spatial', 'nFeature_Spatial')){
        plot <- plot + scale_color_gradientn(colours = heatmap_Palette(100))
    }else if(features %in% c('celltype', 'seurat_clusters')){
        plot <- plot + scale_color_manual(values = cluster_Palette) +
                guides(colour = guide_legend(override.aes = list(size=3), nrow = 4))
    }
    plot <- plot + theme_void()
    return(plot)
}



# 02. standard analysis

In [ ]:
min_gene <- 50
max_gene <- 20000
min_count <-100
max_count <- 40000
mt_cut <- 30
use_pcs_max <- 50; #聚类需要用到前几个PCs


In [ ]:
args <- commandArgs(TRUE)
gemfile <- args[1]  # Input gem file path
prefix <- args[2]   # Prefix for output file
output_dir <- args[3]  # Output file path

seurat_spatialObj=gem_to_seuratObject(gemfile,prefix,20) # bin20
seurat_spatialObj=calc_percenterage_feature_set(seurat_spatialObj)
st_dat=filter_st_dat(seurat_spatialObj, minFeature=min_gene, maxFeature=max_gene, minCount=min_count, maxCount=max_count, minCell=mincell, mtRatio=mt_cut)

seurat_spatialObj@images$image = new(
    Class = 'SlideSeq',
    assay = "Spatial",
    key = "image_",
    coordinates = seurat_spatialObj@meta.data[, c('y', 'x')]
    )
st_dat@images$image = new(
    Class = 'SlideSeq',
    assay = "Spatial",
    key = "image_",
    coordinates = st_dat@meta.data[, c('y', 'x')]
    )

pdf(paste0(output_dir,prefix,".iPlot.pdf"),w=8,h=8)
p1=iPlot(seurat_spatialObj, features="nFeature_Spatial", pt.size = 0.04, cluster_Palette = ColorPalette(50))
p2=iPlot(seurat_spatialObj, features="nCount_Spatial", pt.size = 0.04, cluster_Palette = ColorPalette(50))
p3=iPlot(st_dat, features="nFeature_Spatial", pt.size = 0.04, cluster_Palette = ColorPalette(50))
p4=iPlot(st_dat, features="nCount_Spatial", pt.size = 0.04, cluster_Palette = ColorPalette(50))
multiplot(p1,p2,p3,p4,cols=2)
dev.off()

st_dat <- SCTransform(st_dat, assay = "Spatial", return.only.var.genes =TRUE, n_genes=NULL,do.scale=TRUE,vars.to.regress=c("percent.ribo","percent.mt","nCount_Spatial"), method='qpoisson')
st_dat <- RunPCA(st_dat)
st_dat <- FindNeighbors(st_dat, dims = 1:use_pcs_max)
st_dat <- RunUMAP(st_dat, dims = 1:use_pcs_max)
st_dat <- RunTSNE(st_dat, dims = 1:use_pcs_max,check_duplicates = FALSE)

write.table(st_dat@meta.data,file=paste(output_dir,prefix,".metadata.xls",sep=""),sep="\t",quote=F,row.names=F)
saveRDS(st_dat,file=paste0(output_dir,prefix,".ST.rds"))